<a href="https://colab.research.google.com/github/ravichas/PredAnal4Business_Udacity/blob/main/PredictingCatalogDemand_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Catalog Demand

This project is part of Predictive Analytics for Business Nanodegree. 
Details of the project are explained in the pdf document,   `p1-submission-SRavichandan.pdf`


In [ ]:
install.packages("ggthemes")

In [ ]:
install.packages("readxl")

In [ ]:
install.packages("fastDummies")

In [ ]:
library(readxl)
library(fastDummies)
library(ggthemes)

In [ ]:
p1_customers <- read_excel("p1-customers.xlsx")
#View(p1_customers)  
p1_mailinglist <- read_excel("p1-mailinglist.xlsx")

In [ ]:
colnames(p1_customers)
colnames(p1_mailinglist)


In [ ]:

#clean the colnames 
library(tidyverse)
p1_customers <- p1_customers %>% 
  rename(Num_Years_as_Customer = "#_Years_as_Customer")

p1_mailinglist <- p1_mailinglist %>% 
  rename(Num_Years_as_Customer = "#_Years_as_Customer")

In [ ]:
library(fastDummies)
p1_customers <- dummy_cols(p1_customers, select_columns = "Customer_Segment", remove_first_dummy = TRUE,
                 remove_selected_columns = TRUE)

p1_mailinglist <- dummy_cols(p1_mailinglist, select_columns = "Customer_Segment", remove_first_dummy = TRUE,
                            remove_selected_columns = TRUE)

# EDA

In [ ]:
b <- ggplot(p1_customers, aes(Num_Years_as_Customer, Avg_Sale_Amount)) 
b + geom_point() + 
  geom_smooth(method="lm") +
  ggthemes::theme_economist() +
  ggtitle("Num. of Years as Customer vs Avg Sale Amount");

In [ ]:
b <- ggplot(p1_customers, aes(Avg_Num_Products_Purchased, Avg_Sale_Amount)) 
b + geom_point() + 
  geom_smooth(method="lm") +
  ggthemes::theme_economist() +
  ggtitle("<Num Products purchased> vs Avg Sale Amount")

In [ ]:
ggplot(p1_customers, aes(CS_LC_and_CC, Avg_Sale_Amount)) +
  geom_point() + 
  geom_smooth(method="lm") +
  ggthemes::theme_economist() +
  ggtitle("CS_LC_and_CC = Customer_Segment_Loyalty\n Club and Credit Card");

In [ ]:
ggplot(p1_customers, aes(CS_LC_Only, Avg_Sale_Amount)) +
  geom_point() + 
  geom_smooth(method="lm") +
  ggthemes::theme_economist() +
  ggtitle("CS_LC_Only = Customer_Segment_Loyalty Club Only");

In [ ]:
ggplot(p1_customers, aes(CS_SML, Avg_Sale_Amount)) +
  geom_point() + 
  geom_smooth(method="lm") +
  ggthemes::theme_economist() +
  ggtitle("CS_SML = Customer_Segment_Store Mailing List")

In [ ]:
# note the base level is set to credit card only class
p1_customers <- p1_customers %>% 
  rename(CS_LC_and_CC = "Customer_Segment_Loyalty Club and Credit Card",
         CS_LC_Only = "Customer_Segment_Loyalty Club Only",
         CS_SML = "Customer_Segment_Store Mailing List")


p1_mailinglist <- p1_mailinglist %>% 
  rename(CS_LC_and_CC = "Customer_Segment_Loyalty Club and Credit Card",
         CS_LC_Only = "Customer_Segment_Loyalty Club Only",
         CS_SML = "Customer_Segment_Store Mailing List")

Note, here I use only selected features for modeling. This decision was based on statistical significance. 

In [ ]:
model <- lm(data = p1_customers, Avg_Sale_Amount ~ Avg_Num_Products_Purchased + 
              CS_LC_and_CC + 
              CS_LC_Only + 
              CS_SML)
summary(model)

In [ ]:
p1_mailinglist$Predicted_Average_Sale <- predict(model, p1_mailinglist)
p1_mailinglist_mod <- p1_mailinglist %>% mutate(Avg_Sale = Predicted_Average_Sale * Score_Yes)

p1_mailinglist_mod %>% select(Avg_Sale) %>% select(Avg_Sale) %>% summarise(Avg_Sale_Sum = (sum(Avg_Sale) * 0.5 - 1625.0))